In [0]:
!ls /root

In [0]:
%cd
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools
print("run protoc command")
%cd ~/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
#os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/'

print("run builder_test")
!python object_detection/builders/model_builder_test.py

/root
fatal: destination path 'models' already exists and is not an empty directory.
run protoc command
/root/models/research
run builder_test

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.074s

OK (skipped=1)


**Download ngrok**

In [0]:
%cd
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

/root
--2019-04-04 11:06:24--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.201.75.180, 52.2.175.150, 54.174.228.92, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.201.75.180|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  14.28M  53.2MB/s    in 0.3s    

2019-04-04 11:06:24 (53.2 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
!rm training/ssdlite_mobilenet*

In [0]:
%cd /root/models/research/object_detection
!mkdir training

/root/models/research/object_detection
mkdir: cannot create directory ‘training’: File exists


**Mount gdrive and get necessary data**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/using_flickr.zip .
!unzip using_flickr.zip

Archive:  using_flickr.zip
   creating: images/
  inflating: images/3218467456.jpg   
  inflating: images/3213763363.jpg   
  inflating: images/3202360116.jpg   
  inflating: images/3118428220.jpg   
  inflating: images/372180408.jpg    
  inflating: images/3154099645.jpg   
  inflating: images/3117510822.jpg   
  inflating: images/2960027434.jpg   
  inflating: images/4376175015.jpg   
  inflating: images/3836904950.jpg   
  inflating: images/3752341982.jpg   
  inflating: images/3054222264.jpg   
  inflating: images/2427200136.jpg   
  inflating: images/4731055503.jpg   
  inflating: images/355018646.jpg    
  inflating: images/2838426683.jpg   
  inflating: images/2584787090.jpg   
  inflating: images/144488978.jpg    
  inflating: images/3963201475.jpg   
  inflating: images/3892997631.jpg   
  inflating: images/3624592154.jpg   
  inflating: images/351548830.jpg    
  inflating: images/1597577216.jpg   
  inflating: images/4573390606.jpg   
  inflating: images/4043571759.jpg   
  

In [0]:
!ls -1 images | wc -l

209


In [0]:
!rm training/*


In [0]:
#!cp object-detection.pbtxt training/
#!cp object-detection.pbtxt data/
#!cp faster_rcnn_inception_v2_coco.config training/
!cp ssd_mobilenet_v1_pets.config training/

cp: cannot stat 'ssd_mobilenet_v1_pets.config': No such file or directory


In [0]:
!cp ssd_mobilenet_v1_pets\ \(1\).config ssd_mobilenet_v1_pets.config

In [0]:
!cp my_data/* data/.

In [0]:
%cd training/

/root/models/research/object_detection/training


In [0]:
#Upload dataset from local file system
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  fileName = fn


Saving ssdlite_mobilenet_v2_coco.config to ssdlite_mobilenet_v2_coco.config
User uploaded file "ssdlite_mobilenet_v2_coco.config" with length 4681 bytes


In [0]:
!cat data/object-detection.pbtxt

In [0]:
!cp my_data/* data/.

In [0]:
#Generate TFrecord
!python3 data/generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=data/train.record
!python3 data/generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=data/test.record

In [0]:
%cd training/

/root/models/research/object_detection/training


In [0]:
%cd ~/models/research/object_detection/
import os
import shutil
import glob
import urllib.request
import tarfile

MODEL = 'ssdlite_mobilenet_v2_coco_2018_05_09'
#MODEL = 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03'
#MODEL = 'ssd_mobilenet_v1_coco_2018_01_28'
#MODEL = 'faster_rcnn_resnet101_coco_2018_01_28'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'pretrained_model'

if not (os.path.exists(MODEL_FILE)):
  opener = urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

#os.remove(MODEL_FILE)
#if (os.path.exists(DEST_DIR)):
#  shutil.rmtree(DEST_DIR)
#os.rename(MODEL, DEST_DIR)

/root/models/research/object_detection


In [0]:
%cd ..

/root/models/research/object_detection


In [0]:
!ls ssd* -l


**Setup for Tensorboard**

In [0]:
%cd ../

/root/models/research/object_detection


In [0]:
!kill 684

In [0]:
LOG_DIR = './training'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('~/ngrok http 6006 &')

In [0]:
#!tensorboard --logdir ./training

E0301 07:18:37.255790 MainThread program.py:201] TensorBoard could not bind to port 6006, it was already in use
E0301 07:18:37.255789 140610098816896 program.py:201] TensorBoard could not bind to port 6006, it was already in use
ERROR: TensorBoard could not bind to port 6006, it was already in use


In [0]:
import json, sys
json.load(sys.stdin)['tunnels'][0]['public_url']

In [0]:
#get the public url
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://ae2084f5.ngrok.io


In [0]:
!cp data/ssd_resnet50_v1_fpn.config training/

In [0]:
!ls training/

ssd_mobilenet_v1_pets.config


In [0]:
!rm training/graph.pbtxt

In [0]:
!rm training/pipeline.config

In [0]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.DEBUG)

In [0]:
!cp data/ssd_resnet50_v1_fpn.config training/


In [0]:
!python3 legacy/train.py --logtostderr --train_dir=training --pipeline_config_path=training/ssdlite_mobilenet_v2_coco.config


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use object_detection/model_main.py.
Instructions for updating:
Please switch to tf.train.create_global_step
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions

In [0]:
!python3 legacy/train.py --logtostderr --train_dir=training --pipeline_config_path=training/ssd_resnet50_v1_fpn.config

In [0]:
!ps | grep train

In [0]:
#lst = os.listdir('training_rcnn')
#lf = filter(lambda k: 'model.ckpt-' in k, lst)
#last_model = sorted(lf)[-1].replace('.meta', '')

!python ~/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/root/models/research/object_detection/training/ssd_resnet50_v1_fpn.config \
    --output_directory=fine_tuned_model \
    --trained_checkpoint_prefix=training/model.ckpt-510

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README.md for details
Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes
216 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              0
-min_occurrence             0
-step                       -1
-order_by                   name
-a

In [0]:
last_model

'model.ckpt-19068'

In [0]:
!python -m zipfile -c fine_tuned_model_resnet.zip fine_tuned_model

In [0]:
!ls fine_tuned_model* -l

In [0]:
files.download('fine_tuned_model_part1.zip')

In [0]:
!ls images


In [0]:
%cd ../

/root/models/research/object_detection


**Download the learned model**

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

file_metadata = {
  'name': 'fine_tuned_model_resnet',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('fine_tuned_model_resnet.zip', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1bH9SY_syanjrSMJiVMP0lanmjfUP-XzX


In [0]:
from google.colab import files

files.download('fine_tuned_model_20.zip')

In [0]:
import tensorflow as tf
print(tf.__version__)


1.12.0


**Run Inference**

In [0]:
last_model

'model.ckpt-3534'